<a href="https://colab.research.google.com/github/aioont/KH012-FinWise/blob/main/FinWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Data Preprocessing

In [ ]:
df = pd.read_csv("/content/nsestockhistoricalratios.csv")

#split dataframe on companies
stock = {}
for index, row in df.iterrows():
    key = row['Stock']
    if key not in stock:
        stock[key] = pd.DataFrame(columns=df.columns)
        continue
    stock[key] = pd.concat([stock[key], row.to_frame().T], ignore_index=True)

In [ ]:
#reshaping dataframe
for key in stock:
  stock[key].drop(['Stock'], axis=1, inplace=True)
  company = pd.pivot_table(stock[key], index="Year")

  #min-max scaling
  for col in company.columns:
    normalized_values = (company[col] - np.min(company[col])) / (np.max(company[col]) - np.min(company[col]))
    company.loc[:, col] = normalized_values

  #Delete rows and fill columns with 0 for missing data
  company = company.dropna()
  #company = company.dropna(axis=1)

  stock[key] = company
  file_name = f"forheatmap_{key}.csv"
  stock[key].to_csv(file_name, index=False)

<ipython-input-45-4fd67415897f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  company.loc[:, col] = normalized_values


In [ ]:
for key in stock:
  file_name = f"/content/forheatmap_{key}.csv"
  na = [-99, 99, 'NULL', '']
  stock[key] = pd.read_csv(file_name, na_values=na)

  print(stock[key].isna().sum().sum())

In [ ]:
#Creating Heatmaps for each company
for key in list(stock.keys()):
  stock[key].drop(['Stock'], axis=1, inplace=True)
  company = pd.pivot_table(stock[key], index="Year")

  #min-max scaling
  for col in company.columns:
    company[col] = (company[col] - np.min(company[col])) / (np.max(company[col]) - np.min(company[col]))

  sns.heatmap(company, cmap="YlGnBu", fmt="0.0f")
  # filename = f"heatmap_{key}.png"
  # plt.savefig(filename)

Model Architecture

In [ ]:
ClassifyStock = Sequential([
    Conv2D(64, (58,5), strides=(5,5), activation='relu', input_size=(640,480,3)),
    MaxPool2D((5,5)),

    Conv2D(64, (58,5), strides=(5,5), activation='relu', input_size=(640,480,3)),
    MaxPool2D((5,5)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

Model Compile

In [ ]:
ClassifyStock.compile(loss='', optimizer='', metrics='')